In [66]:
# Kernel alignment check
import sys
print('python executable:', sys.executable)
try:
    import folium
    print('folium', folium.__version__)
except Exception as e:
    print('folium import error:', e)

python executable: c:\Users\User\Documents\V-lib_Proj\.venv\Scripts\python.exe
folium 0.20.0


In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
df = pd.read_csv("comptage-velo-donnees-compteurs (2).csv", sep=";")

In [69]:
delete_columns = ['Lien vers photo du site de comptage', 'ID Photos', 'test_lien_vers_photos_du_site_de_comptage_', 'id_photo_1', 'url_sites', 'type_dimage']
df = df.drop(delete_columns, axis = 1)

In [70]:
# Sample function to transform column names
def transform_column_name(name):
    return name.replace(' ', '_').lower()

# Apply to all columns of the DataFrame
df.columns = [transform_column_name(col) for col in df.columns]

In [71]:
display(df.head())

,identifiant_du_compteur,nom_du_compteur,identifiant_du_site_de_comptage,nom_du_site_de_comptage,comptage_horaire,date_et_heure_de_comptage,date_d'installation_du_site_de_comptage,coordonnées_géographiques,identifiant_technique_compteur,mois_annee_comptage
0,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01T05:00:00+02:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08
1,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01T08:00:00+02:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08
2,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01T10:00:00+02:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08
3,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01T06:00:00+02:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08
4,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01T14:00:00+02:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899336 entries, 0 to 899335
Data columns (total 10 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   identifiant_du_compteur                  872571 non-null  object 
 1   nom_du_compteur                          899336 non-null  object 
 2   identifiant_du_site_de_comptage          872571 non-null  float64
 3   nom_du_site_de_comptage                  872571 non-null  object 
 4   comptage_horaire                         899336 non-null  int64  
 5   date_et_heure_de_comptage                899336 non-null  object 
 6   date_d'installation_du_site_de_comptage  872571 non-null  object 
 7   coordonnées_géographiques                872571 non-null  object 
 8   identifiant_technique_compteur           863338 non-null  object 
 9   mois_annee_comptage                      899336 non-null  object 
dtypes: float64(1), int64(1), object(

In [73]:
#enlever les NA
df = df.dropna()

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 863338 entries, 0 to 899335
Data columns (total 10 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   identifiant_du_compteur                  863338 non-null  object 
 1   nom_du_compteur                          863338 non-null  object 
 2   identifiant_du_site_de_comptage          863338 non-null  float64
 3   nom_du_site_de_comptage                  863338 non-null  object 
 4   comptage_horaire                         863338 non-null  int64  
 5   date_et_heure_de_comptage                863338 non-null  object 
 6   date_d'installation_du_site_de_comptage  863338 non-null  object 
 7   coordonnées_géographiques                863338 non-null  object 
 8   identifiant_technique_compteur           863338 non-null  object 
 9   mois_annee_comptage                      863338 non-null  object 
dtypes: float64(1), int64(1), object(8)
me

In [75]:
# Parse the datetime column correctly
df['date_et_heure_de_comptage'] = pd.to_datetime(df['date_et_heure_de_comptage'], utc=True)

# Remove the timezone (make tz-naive to avoid time zone issues (mismatches))
df['date_et_heure_de_comptage'] = df['date_et_heure_de_comptage'].dt.tz_convert(None)

In [76]:
# Function to determine the season from a date
def get_season_from_date(date):
    year = date.year
    spring = pd.Timestamp(f'{year}-03-20')
    summer = pd.Timestamp(f'{year}-06-21')
    autumn = pd.Timestamp(f'{year}-09-22')
    winter = pd.Timestamp(f'{year}-12-21')

    if spring <= date < summer:
        return 'spring'
    elif summer <= date < autumn:
        return 'summer'
    elif autumn <= date < winter:
        return 'autumn'
    else:
        return 'winter'

# Apply to DataFrame
df['saison'] = df['date_et_heure_de_comptage'].apply(get_season_from_date)

In [ ]:
# Define the list of holiday intervals (inclusive start, exclusive end)
vacances_periods = [
    ('2024-10-19', '2024-11-05'),  # Toussaint
    ('2024-12-21', '2025-01-07'),  # Noël
    ('2025-02-15', '2025-03-04'),  # Hiver
    ('2025-04-12', '2025-04-29'),  # Printemps
    ('2025-05-29', '2025-06-01'),  # Ascension + pont (29, 30, 31)
    ('2025-07-05', '2025-09-02'),  # Summer begins 5 July to 1 Sept
]

# Convert to datetime timestamps
vacances_intervals = [
    (pd.Timestamp(start), pd.Timestamp(end))
    for start, end in vacances_periods
]

# Define function to test if date falls in a holiday
def is_vacances(date):
    for start, end in vacances_intervals:
        if start <= date < end:
            return True
    return False

# Apply to DataFrame
df['vacances'] = df['date_et_heure_de_comptage'].apply(is_vacances)

In [78]:
# Function to classify rush hour
def is_rush_hour(dt):
    hour = dt.hour
    return (7 <= hour < 10) or (17 <= hour < 20)

# Apply to create the new column
df['heure_de_pointe'] = df['date_et_heure_de_comptage'].apply(is_rush_hour)

In [79]:
# Find the anchor date (the earliest datetime)
anchor_date = df['date_et_heure_de_comptage'].min().normalize()  # Drop time part

print(anchor_date) # This was a Saturday based on a Google search

df['jour'] = df['date_et_heure_de_comptage'].apply(
    lambda x: ((x.normalize() - anchor_date).days + 5) % 7
)

2024-08-01 00:00:00


In [80]:
# Créer un dictionnaire pour mapper les chiffres aux jours de la semaine
jours_de_la_semaine = {
    0: 'Samedi',
    1: 'Dimanche',
    2: 'Lundi',
    3: 'Mardi',
    4: 'Mercredi',
    5: 'Jeudi',
    6: 'Vendredi'
}

df['nom_jour'] = df['jour'].map(jours_de_la_semaine)

In [81]:
df["heure"] = df["date_et_heure_de_comptage"].dt.hour

In [82]:
# Example rough night hours per season (24h format)
night_hours = {
    'winter':    {'start': 17, 'end': 8},
    'spring':{'start': 20.5, 'end': 6},   # 20:30
    'summer':      {'start': 22, 'end': 5},
    'autumn':  {'start': 19, 'end': 7},
}

def is_night(row):
    season = row['saison'].lower()
    dt = row['date_et_heure_de_comptage']
    hour = dt.hour + dt.minute/60  # fractional hour
    
    nh = night_hours.get(season)
    if nh is None:
        # if season is unknown, consider not night
        return False
    
    start, end = nh['start'], nh['end']
    
    # Since all seasons cross midnight, we only need this check
    return hour >= start or hour < end

# Apply to DataFrame
df['nuit'] = df.apply(is_night, axis=1)

In [83]:
# Séparer la colonne "coordonnees_geographiques" en deux colonnes latitude et longitude
df[["latitude", "longitude"]] = df["coordonnées_géographiques"].str.split(",", expand=True)

df["latitude"] = df["latitude"].astype(float)
df["longitude"] = df["longitude"].astype(float)

In [84]:
#vérification
df.head()

,identifiant_du_compteur,nom_du_compteur,identifiant_du_site_de_comptage,nom_du_site_de_comptage,comptage_horaire,date_et_heure_de_comptage,date_d'installation_du_site_de_comptage,coordonnées_géographiques,identifiant_technique_compteur,mois_annee_comptage,saison,vacances,heure_de_pointe,jour,nom_jour,heure,nuit,latitude,longitude
0,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 03:00:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08,summer,False,False,5,Jeudi,3,True,48.83507,2.33305
1,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 06:00:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08,summer,False,False,5,Jeudi,6,False,48.83507,2.33305
2,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 08:00:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08,summer,False,True,5,Jeudi,8,False,48.83507,2.33305
3,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 04:00:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08,summer,False,False,5,Jeudi,4,True,48.83507,2.33305
4,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 12:00:00,2012-02-22,"48.83507,2.33305",Y2H20114504,2024-08,summer,False,False,5,Jeudi,12,False,48.83507,2.33305


In [85]:
df = df.drop(columns=["coordonnées_géographiques"])

In [86]:
# Créer une version unique pour l'affichage, sans modifier le df original
df_unique = df.drop_duplicates(subset=["nom_du_site_de_comptage"])

In [87]:
import folium

m = folium.Map(location=(48.86, 2.33), zoom_start=13)

for _, row in df_unique.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row["nom_du_site_de_comptage"],
        icon=folium.Icon(color="blue", icon="bicycle", prefix="fa")
    ).add_to(m)

m.save("compteur_paris.html")

In [88]:
import pandas as pd

def get_date_range(df, date_col):
    min_date = df[date_col].min()
    max_date = df[date_col].max()
    
    return min_date, max_date


In [89]:
min_date, max_date = get_date_range(df, "date_et_heure_de_comptage")

print("Date la plus ancienne :", min_date)
print("Date la plus récente  :", max_date)

Date la plus ancienne : 2024-08-01 03:00:00
Date la plus récente  : 2025-09-02 21:00:00


In [90]:
#API météo

import requests
import pandas as pd
# API endpoint
url = "https://archive-api.open-meteo.com/v1/archive?latitude=48.8575&longitude=2.3514&start_date=2024-08-01&end_date=2025-10-07&hourly=rain,snowfall,apparent_temperature,wind_speed_10m"
# Send GET request
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    # print(data)
    # records = data.get("temperature_2m", [])
    # temperature = data.get("temperature_2m", [])
    records = data["hourly"]
    # Convert list of dicts to DataFrame
    df_weather = pd.DataFrame(records)
    
    # Show first few rows
    print(df_weather)
else:
    print("Error:", response.status_code)

                   time  rain  snowfall  apparent_temperature  wind_speed_10m
0      2024-08-01T00:00   0.0       0.0                  25.6             4.6
1      2024-08-01T01:00   0.2       0.0                  25.5            11.7
2      2024-08-01T02:00   9.9       0.0                  23.3            13.7
3      2024-08-01T03:00   0.8       0.0                  24.2             6.6
4      2024-08-01T04:00   8.2       0.0                  23.0             6.4
...                 ...   ...       ...                   ...             ...
10387  2025-10-07T19:00   0.0       0.0                  15.1             2.7
10388  2025-10-07T20:00   0.0       0.0                  14.5             1.7
10389  2025-10-07T21:00   0.0       0.0                  14.0             1.8
10390  2025-10-07T22:00   0.0       0.0                  13.0             3.4
10391  2025-10-07T23:00   0.0       0.0                  12.6             2.1

[10392 rows x 5 columns]


In [91]:
df_weather['time'] = pd.to_datetime(df_weather['time'], utc=True)
df_weather['time'] = df_weather['time'].dt.tz_convert(None)
print(df_weather)


                     time  rain  snowfall  apparent_temperature  \
0     2024-08-01 00:00:00   0.0       0.0                  25.6   
1     2024-08-01 01:00:00   0.2       0.0                  25.5   
2     2024-08-01 02:00:00   9.9       0.0                  23.3   
3     2024-08-01 03:00:00   0.8       0.0                  24.2   
4     2024-08-01 04:00:00   8.2       0.0                  23.0   
...                   ...   ...       ...                   ...   
10387 2025-10-07 19:00:00   0.0       0.0                  15.1   
10388 2025-10-07 20:00:00   0.0       0.0                  14.5   
10389 2025-10-07 21:00:00   0.0       0.0                  14.0   
10390 2025-10-07 22:00:00   0.0       0.0                  13.0   
10391 2025-10-07 23:00:00   0.0       0.0                  12.6   

       wind_speed_10m  
0                 4.6  
1                11.7  
2                13.7  
3                 6.6  
4                 6.4  
...               ...  
10387             2.7  
103

In [92]:
df_merged = pd.merge(
    df,
    df_weather,
    how="left",
    left_on="date_et_heure_de_comptage",
    right_on="time"
).drop(columns=["time"])

In [93]:
df_merged.head()


,identifiant_du_compteur,nom_du_compteur,identifiant_du_site_de_comptage,nom_du_site_de_comptage,comptage_horaire,date_et_heure_de_comptage,date_d'installation_du_site_de_comptage,identifiant_technique_compteur,mois_annee_comptage,saison,...,jour,nom_jour,heure,nuit,latitude,longitude,rain,snowfall,apparent_temperature,wind_speed_10m
0,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 03:00:00,2012-02-22,Y2H20114504,2024-08,summer,...,5,Jeudi,3,True,48.83507,2.33305,0.8,0.0,24.2,6.6
1,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 06:00:00,2012-02-22,Y2H20114504,2024-08,summer,...,5,Jeudi,6,False,48.83507,2.33305,0.0,0.0,22.9,10.4
2,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 08:00:00,2012-02-22,Y2H20114504,2024-08,summer,...,5,Jeudi,8,False,48.83507,2.33305,0.0,0.0,26.8,8.0
3,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 04:00:00,2012-02-22,Y2H20114504,2024-08,summer,...,5,Jeudi,4,True,48.83507,2.33305,8.2,0.0,23.0,6.4
4,100003098-101003098,106 avenue Denfert Rochereau NE-SO,100003098.0,106 avenue Denfert Rochereau,0,2024-08-01 12:00:00,2012-02-22,Y2H20114504,2024-08,summer,...,5,Jeudi,12,False,48.83507,2.33305,0.0,0.0,31.7,12.6


In [94]:
df_merged.shape
df_merged.info()
df_merged.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863338 entries, 0 to 863337
Data columns (total 22 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   identifiant_du_compteur                  863338 non-null  object        
 1   nom_du_compteur                          863338 non-null  object        
 2   identifiant_du_site_de_comptage          863338 non-null  float64       
 3   nom_du_site_de_comptage                  863338 non-null  object        
 4   comptage_horaire                         863338 non-null  int64         
 5   date_et_heure_de_comptage                863338 non-null  datetime64[ns]
 6   date_d'installation_du_site_de_comptage  863338 non-null  object        
 7   identifiant_technique_compteur           863338 non-null  object        
 8   mois_annee_comptage                      863338 non-null  object        
 9   saison                    

identifiant_du_compteur                            object
nom_du_compteur                                    object
identifiant_du_site_de_comptage                   float64
nom_du_site_de_comptage                            object
comptage_horaire                                    int64
date_et_heure_de_comptage                  datetime64[ns]
date_d'installation_du_site_de_comptage            object
identifiant_technique_compteur                     object
mois_annee_comptage                                object
saison                                             object
vacances                                             bool
heure_de_pointe                                      bool
jour                                                int64
nom_jour                                           object
heure                                               int32
nuit                                                 bool
latitude                                          float64
longitude     

In [95]:
df_merged['pluie'] = (df_merged['rain'] > 0)
df_merged.groupby('pluie')['comptage_horaire'].mean()

#la pluie n'influence pas forcément l'affluence en vélo

pluie
False    80.491664
True     69.419103
Name: comptage_horaire, dtype: float64

In [96]:
df_merged['vent'] = (df_merged['wind_speed_10m'] > 30)
df_merged.groupby('vent')['comptage_horaire'].mean()

#le vent n'influence pas forcément l'affluence en vélo

vent
False    78.830986
True     64.951122
Name: comptage_horaire, dtype: float64

In [97]:
df_merged['neige'] = (df_merged['snowfall'] > 0)
df_merged.groupby('neige')['comptage_horaire'].mean()

#la neige influe un minimum mais pas forcément significatif 

neige
False    78.881715
True     53.884663
Name: comptage_horaire, dtype: float64

In [98]:
print(df_merged.columns)

Index(['identifiant_du_compteur', 'nom_du_compteur',
       'identifiant_du_site_de_comptage', 'nom_du_site_de_comptage',
       'comptage_horaire', 'date_et_heure_de_comptage',
       'date_d'installation_du_site_de_comptage',
       'identifiant_technique_compteur', 'mois_annee_comptage', 'saison',
       'vacances', 'heure_de_pointe', 'jour', 'nom_jour', 'heure', 'nuit',
       'latitude', 'longitude', 'rain', 'snowfall', 'apparent_temperature',
       'wind_speed_10m', 'pluie', 'vent', 'neige'],
      dtype='object')


In [99]:
df_merged["mois"] = df_merged["date_et_heure_de_comptage"].dt.month

In [100]:
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863338 entries, 0 to 863337
Data columns (total 26 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   identifiant_du_compteur                  863338 non-null  object        
 1   nom_du_compteur                          863338 non-null  object        
 2   identifiant_du_site_de_comptage          863338 non-null  float64       
 3   nom_du_site_de_comptage                  863338 non-null  object        
 4   comptage_horaire                         863338 non-null  int64         
 5   date_et_heure_de_comptage                863338 non-null  datetime64[ns]
 6   date_d'installation_du_site_de_comptage  863338 non-null  object        
 7   identifiant_technique_compteur           863338 non-null  object        
 8   mois_annee_comptage                      863338 non-null  object        
 9   saison                    

In [101]:
print(df_merged["date_et_heure_de_comptage"])

0        2024-08-01 03:00:00
1        2024-08-01 06:00:00
2        2024-08-01 08:00:00
3        2024-08-01 04:00:00
4        2024-08-01 12:00:00
                 ...        
863333   2025-09-02 12:00:00
863334   2025-09-02 13:00:00
863335   2025-09-02 19:00:00
863336   2025-09-02 20:00:00
863337   2025-09-02 21:00:00
Name: date_et_heure_de_comptage, Length: 863338, dtype: datetime64[ns]


In [102]:
print(df_merged["identifiant_du_compteur"].nunique())
print(df_merged["nom_du_compteur"].nunique())
print(df_merged["nom_du_site_de_comptage"].nunique())
print(df_merged["identifiant_du_site_de_comptage"].nunique()) # Does not exactly coincide with nom_du_site_de_comptage, why???


97
97
65
68


In [103]:
# Dynamically aggregate: "mean" for comptage_horaire, "first" for others
agg_dict = {col: "first" for col in df_merged.columns if col not in ["identifiant_du_site_de_comptage", "date_et_heure_de_comptage", "comptage_horaire"]}
agg_dict["comptage_horaire"] = "mean"

df_merged = df_merged.groupby(
    ["identifiant_du_site_de_comptage", "date_et_heure_de_comptage"],
    as_index=False
).agg(agg_dict)

# df_merged = df_merged.groupby(
#     ["identifiant_du_site_de_comptage", "date_et_heure_de_comptage"],
#     as_index=False
# ).agg({
#     "comptage_horaire": "mean",
#     "identifiant_du_compteur": "first",
#     "nom_du_compteur": "first",
#     "nom_du_site_de_comptage": "first",
#     "date_d'installation_du_site_de_comptage": "first",
#     "identifiant_technique_compteur": "first",
#     "mois_annee_comptage": "first",
#     "saison": "first",
#     "vacances": "first",
#     "heure_de_pointe": "first",
#     "jour": "first",
#     "nom_jour": "first",
#     "heure": "first",
#     "nuit": "first",
#     "latitude": "first",
#     "longitude": "first"
# })

In [104]:
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614744 entries, 0 to 614743
Data columns (total 26 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   identifiant_du_site_de_comptage          614744 non-null  float64       
 1   date_et_heure_de_comptage                614744 non-null  datetime64[ns]
 2   identifiant_du_compteur                  614744 non-null  object        
 3   nom_du_compteur                          614744 non-null  object        
 4   nom_du_site_de_comptage                  614744 non-null  object        
 5   date_d'installation_du_site_de_comptage  614744 non-null  object        
 6   identifiant_technique_compteur           614744 non-null  object        
 7   mois_annee_comptage                      614744 non-null  object        
 8   saison                                   614744 non-null  object        
 9   vacances                  

In [105]:
# Removing unnecessary columns for modeling
df_merged = df_merged.drop(columns=["latitude", "longitude","date_d'installation_du_site_de_comptage", \
                                    "identifiant_technique_compteur", "mois_annee_comptage", "identifiant_du_compteur", \
                                        "nom_du_site_de_comptage", "jour", "nom_du_compteur", "snowfall", "rain", "vent", \
                                            "wind_speed_10m"])

In [106]:
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614744 entries, 0 to 614743
Data columns (total 13 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   identifiant_du_site_de_comptage  614744 non-null  float64       
 1   date_et_heure_de_comptage        614744 non-null  datetime64[ns]
 2   saison                           614744 non-null  object        
 3   vacances                         614744 non-null  bool          
 4   heure_de_pointe                  614744 non-null  bool          
 5   nom_jour                         614744 non-null  object        
 6   heure                            614744 non-null  int32         
 7   nuit                             614744 non-null  bool          
 8   apparent_temperature             614744 non-null  float64       
 9   pluie                            614744 non-null  bool          
 10  neige                            614744 non-

In [107]:
# I am not going to train one model per identifiant_du_site_de_comptage as it would be too computationally complex
# Therefore, I will compute some statistics per site and use them as additional features (static features, i.e. not changing over time) first
# to capture site-specific usage patterns and make up for the lack of per-site models
# Next, I will compute time-varying features per site (e.g., rolling averages) to capture temporal patterns specific to each site with the 
# goal of improving model accuracy without the overhead of multiple models

site_stats = (
    df_merged.groupby('identifiant_du_site_de_comptage')['comptage_horaire']
    .agg(['mean', 'std', 'max', 'min'])
    .rename(columns={
        'mean': 'site_mean_usage',
        'std': 'site_usage_variability',
        'max': 'site_max_usage',
        'min': 'site_min_usage'
    })
)


In [108]:
df_merged = df_merged.merge(site_stats, on='identifiant_du_site_de_comptage', how='left')

In [109]:
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614744 entries, 0 to 614743
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   identifiant_du_site_de_comptage  614744 non-null  float64       
 1   date_et_heure_de_comptage        614744 non-null  datetime64[ns]
 2   saison                           614744 non-null  object        
 3   vacances                         614744 non-null  bool          
 4   heure_de_pointe                  614744 non-null  bool          
 5   nom_jour                         614744 non-null  object        
 6   heure                            614744 non-null  int32         
 7   nuit                             614744 non-null  bool          
 8   apparent_temperature             614744 non-null  float64       
 9   pluie                            614744 non-null  bool          
 10  neige                            614744 non-

In [110]:
# Time-varying features per site
df_merged = df_merged.sort_values(['identifiant_du_site_de_comptage', 'date_et_heure_de_comptage'])

df_merged['lag_1'] = df_merged.groupby('identifiant_du_site_de_comptage')['comptage_horaire'].shift(1)
df_merged['lag_24'] = df_merged.groupby('identifiant_du_site_de_comptage')['comptage_horaire'].shift(24)
df_merged['rolling_mean_24'] = (
    df_merged.groupby('identifiant_du_site_de_comptage')['comptage_horaire']
    .shift(1).rolling(24).mean()
)


In [111]:
df_merged = df_merged.dropna()

In [112]:
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
Index: 613112 entries, 24 to 614743
Data columns (total 20 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   identifiant_du_site_de_comptage  613112 non-null  float64       
 1   date_et_heure_de_comptage        613112 non-null  datetime64[ns]
 2   saison                           613112 non-null  object        
 3   vacances                         613112 non-null  bool          
 4   heure_de_pointe                  613112 non-null  bool          
 5   nom_jour                         613112 non-null  object        
 6   heure                            613112 non-null  int32         
 7   nuit                             613112 non-null  bool          
 8   apparent_temperature             613112 non-null  float64       
 9   pluie                            613112 non-null  bool          
 10  neige                            613112 non-null

# Encoding Categorical Variables
We'll encode:
- Categorical variables using `pd.get_dummies()`:
  - `saison` (winter, spring, summer, autumn)
  - `nom_jour` (days of week)
- Boolean variables to 0/1:
  - `nuit`, `vacances`, `heure_de_pointe`
  - `pluie`, `neige`, `vent`, `temperature`

In [113]:
# Create a copy of df_merged to avoid modifying the original
df_encoded = df_merged.copy()

# 1. Encode boolean columns to 0/1 (they're already boolean but let's make them int)
boolean_cols = ['nuit', 'vacances', 'heure_de_pointe', 'pluie', 'neige']
for col in boolean_cols:
    df_encoded[col] = df_encoded[col].astype(int)

# 2. Create dummy variables for categorical columns
# Saison (keep n-1 categories to avoid multicollinearity)
saison_dummies = pd.get_dummies(df_encoded['saison'], prefix='saison', drop_first=True)
df_encoded = pd.concat([df_encoded, saison_dummies], axis=1)

# Nom_jour (keep n-1 categories to avoid multicollinearity)
jour_dummies = pd.get_dummies(df_encoded['nom_jour'], prefix='jour', drop_first=True)
df_encoded = pd.concat([df_encoded, jour_dummies], axis=1)

# Mois (keep n-1 categories to avoid multicollinearity)
mois_dummies = pd.get_dummies(df_encoded['mois'], prefix='mois', drop_first=True)
df_encoded = pd.concat([df_encoded, mois_dummies], axis=1)

# Heure (keep n-1 categories to avoid multicollinearity)
heure_dummies = pd.get_dummies(df_encoded['heure'], prefix='heure', drop_first=True)
df_encoded = pd.concat([df_encoded, heure_dummies], axis=1)

# 3. Drop the original categorical columns
df_encoded = df_encoded.drop(['saison', 'nom_jour', "mois", "heure"], axis=1)

# Show the new encoded columns
print("\nNew encoded boolean columns (first few rows):")
print(df_encoded[boolean_cols].head())
print("\nNew encoded categorical columns (first few rows):")
print(df_encoded.filter(like='saison_').head())
print("\nNew encoded day columns (first few rows):")
print(df_encoded.filter(like='jour_').head())


New encoded boolean columns (first few rows):
    nuit  vacances  heure_de_pointe  pluie  neige
24     1         0                0      1      0
25     1         0                0      0      0
26     0         0                0      0      0
27     0         0                0      0      0
28     0         0                1      0      0

New encoded categorical columns (first few rows):
    saison_spring  saison_summer  saison_winter
24          False           True          False
25          False           True          False
26          False           True          False
27          False           True          False
28          False           True          False

New encoded day columns (first few rows):
    jour_Jeudi  jour_Lundi  jour_Mardi  jour_Mercredi  jour_Samedi  \
24       False       False       False          False        False   
25       False       False       False          False        False   
26       False       False       False          False        

In [114]:
import pandas as pd

# 1️⃣ Ensure your datetime column is a datetime type
# This is necessary for TimeSeriesSplit to work properly
df_encoded['date_et_heure_de_comptage'] = pd.to_datetime(df_encoded['date_et_heure_de_comptage'])

# 2️⃣ Sort by time (and optionally station if you have multiple)
# Sorting by both ensures chronological order *within each site*
df_encoded = df_encoded.sort_values(
    by=['date_et_heure_de_comptage', 'identifiant_du_site_de_comptage'],
    ascending=[True, True]
).reset_index(drop=True)

# 3️⃣ Optional sanity checks
print("Earliest timestamp:", df_encoded['date_et_heure_de_comptage'].min())
print("Latest timestamp:", df_encoded['date_et_heure_de_comptage'].max())
print("Number of rows:", len(df_encoded))
print("Number of stations:", df_encoded['identifiant_du_site_de_comptage'].nunique())

# Check if there are duplicates in time per site (which can mess up lags)
dupes = (
    df_encoded
    .groupby(['identifiant_du_site_de_comptage', 'date_et_heure_de_comptage'])
    .size()
    .reset_index(name='count')
    .query('count > 1')
)

if len(dupes) > 0:
    print(f"⚠️ Found {len(dupes)} duplicated (site, datetime) entries — consider cleaning.")
else:
    print("✅ No duplicated timestamps per site found.")

# Optional: ensure the data is evenly spaced (e.g., hourly)
# This will help you spot missing hours per site.
check_freq = (
    df_encoded
    .groupby('identifiant_du_site_de_comptage')['date_et_heure_de_comptage']
    .diff()
    .value_counts()
    .head()
)
print("\nMost common time difference between observations:")
print(check_freq)


Earliest timestamp: 2024-08-02 03:00:00
Latest timestamp: 2025-09-02 21:00:00
Number of rows: 613112
Number of stations: 68
✅ No duplicated timestamps per site found.

Most common time difference between observations:
date_et_heure_de_comptage
0 days 01:00:00    608734
0 days 05:00:00      2190
0 days 04:00:00       866
0 days 03:00:00       697
0 days 02:00:00       486
Name: count, dtype: int64


In [115]:
print(df_encoded["date_et_heure_de_comptage"].max())

2025-09-02 21:00:00


In [116]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd

# Separate features/target
features = [col for col in df_encoded.columns if col not in ['comptage_horaire', 'date_et_heure_de_comptage']]
X = df_encoded[features].reset_index(drop=True)
y = df_encoded['comptage_horaire'].reset_index(drop=True)

# Define time series split
tscv = TimeSeriesSplit(n_splits=5, test_size=5000)  
# You can specify `test_size` or let it infer from n_splits


In [117]:
results = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    print(f"\nFold {fold}: Train={len(X_train)}, Val={len(X_val)}")

    model = LGBMRegressor(n_estimators=200, learning_rate=0.05)
    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    rmse = mean_squared_error(y_val, preds)

    print(f"MAE={mae:.2f}, RMSE={rmse:.2f}")
    results.append({'fold': fold, 'MAE': mae, 'RMSE': rmse})



Fold 1: Train=588112, Val=5000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025927 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1391
[LightGBM] [Info] Number of data points in the train set: 588112, number of used features: 57
[LightGBM] [Info] Start training from score 79.775148
MAE=13.44, RMSE=425.24

Fold 2: Train=593112, Val=5000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1391
[LightGBM] [Info] Number of data points in the train set: 593112, number of used features: 57
[LightGBM] [Info] Start training from score 79.635501
MAE=14.53, RMSE=532.13

Fold 3: Train=598112, Val=5000
[LightGBM] [Info] Auto-choosing row-wise m

In [118]:
cv_results = pd.DataFrame(results)
print("\nAverage CV performance:")
print(cv_results[['MAE', 'RMSE']].mean())



Average CV performance:
MAE      17.274183
RMSE    919.840824
dtype: float64


In [119]:
full_data = df_encoded[features].reset_index(drop=True)
preds = model.predict(full_data)


In [120]:
print(full_data.head())

   identifiant_du_site_de_comptage  vacances  heure_de_pointe  nuit  \
0                      100003098.0         0                0     1   
1                      100006300.0         0                0     1   
2                      100007049.0         0                0     1   
3                      100036718.0         0                0     1   
4                      100036719.0         0                0     1   

   apparent_temperature  pluie  neige  site_mean_usage  \
0                  20.9      1      0         0.000000   
1                  20.9      1      0        68.694342   
2                  20.9      1      0        37.458146   
3                  20.9      1      0        68.844646   
4                  20.9      1      0        37.526144   

   site_usage_variability  site_max_usage  ...  heure_14  heure_15  heure_16  \
0                0.000000             0.0  ...     False     False     False   
1               62.304320           446.0  ...     False     Fal

In [121]:
print(df[["latitude", "longitude", "identifiant_du_site_de_comptage"]])

         latitude  longitude  identifiant_du_site_de_comptage
0       48.835070   2.333050                      100003098.0
1       48.835070   2.333050                      100003098.0
2       48.835070   2.333050                      100003098.0
3       48.835070   2.333050                      100003098.0
4       48.835070   2.333050                      100003098.0
...           ...        ...                              ...
899331  48.837055   2.274019                      300030271.0
899332  48.837055   2.274019                      300030271.0
899333  48.837055   2.274019                      300030271.0
899334  48.837055   2.274019                      300030271.0
899335  48.837055   2.274019                      300030271.0

[863338 rows x 3 columns]


In [122]:
# get unique coords per site and merge into full_data
coords = df[["identifiant_du_site_de_comptage", "latitude", "longitude"]].drop_duplicates(subset="identifiant_du_site_de_comptage")
full_data_geo = full_data.merge(coords, on="identifiant_du_site_de_comptage", how="left")

# quick check
print(full_data_geo[["identifiant_du_site_de_comptage", "latitude", "longitude"]])

        identifiant_du_site_de_comptage   latitude  longitude
0                           100003098.0  48.835070   2.333050
1                           100006300.0  48.843435   2.383378
2                           100007049.0  48.846130   2.375590
3                           100036718.0  48.834360   2.377000
4                           100036719.0  48.853720   2.357020
...                                 ...        ...        ...
613107                      300030173.0  48.821204   2.368576
613108                      300030219.0  48.869220   2.408714
613109                      300030250.0  48.834624   2.405283
613110                      300030271.0  48.837055   2.274019
613111                      300030279.0  48.837006   2.274824

[613112 rows x 3 columns]


In [123]:
print(full_data)

        identifiant_du_site_de_comptage  vacances  heure_de_pointe  nuit  \
0                           100003098.0         0                0     1   
1                           100006300.0         0                0     1   
2                           100007049.0         0                0     1   
3                           100036718.0         0                0     1   
4                           100036719.0         0                0     1   
...                                 ...       ...              ...   ...   
613107                      300030173.0         0                0     0   
613108                      300030219.0         0                0     0   
613109                      300030250.0         0                0     0   
613110                      300030271.0         0                0     0   
613111                      300030279.0         0                0     0   

        apparent_temperature  pluie  neige  site_mean_usage  \
0                       

In [124]:
print(preds)

[-0.84261339  2.89477635  4.47830482 ... 51.04169754 49.79599405
 29.5620204 ]


In [125]:
full_data_geo[full_data_geo["identifiant_du_site_de_comptage"] == 100003098.0]["longitude"].unique() # Check latitude too

array([2.33305])

In [126]:
full_data_geo["predicted_comptage_horaire"] = preds

In [128]:
from folium.plugins import HeatMap

m = folium.Map(location=[48.8566, 2.3522], zoom_start=12, tiles='CartoDB positron')

df_heat = full_data_geo.groupby(['latitude', 'longitude'], as_index=False)['predicted_comptage_horaire'].sum()

# Ajout de la heatmap
HeatMap(
    data=df_heat[['latitude', 'longitude', 'predicted_comptage_horaire']].values,
    radius=30,
    max_zoom=13
).add_to(m)

print(f"This is the heatmap of predicted bike counts in Paris based on the trained model.")
print(f"Each point's intensity reflects the total predicted bike counts at that location.")
print(f"My model predicts at H+1 based on the latest data collected at: {df_encoded["date_et_heure_de_comptage"].max()}.")
m.save("heatmap_velos_paris.html")
m

This is the heatmap of predicted bike counts in Paris based on the trained model.
Each point's intensity reflects the total predicted bike counts at that location.
My model predicts at H+1 based on the latest data collected at: 2025-09-02 21:00:00.
